# Deep learning classification using a LTSM FCN

This  notebook achieves a 95.1% score on the public data set https://www.xstarter.xyz/challenge-details/gamma-log-facies-identification

## Importing packages

In [ ]:
import pandas as pd
import numpy as np 
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.metrics import balanced_accuracy_score, accuracy_score,classification_report
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout, Flatten
from keras.layers import Input, Dense, LSTM, CuDNNLSTM, concatenate, Activation, GRU, SimpleRNN
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
K.tensorflow_backend._get_available_gpus()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## LSTM FCN

IThe code comes from https://github.com/titu1994/LSTM-FCN, which is an implementation of the paper "LSTM Fully Convolutional Networks for Time Series Classification" (https://ieeexplore.ieee.org/document/8141873)

In [ ]:
def generate_lstmfcn(MAX_SEQUENCE_LENGTH, NB_CLASS, NUM_CELLS=8):

    ip = Input(shape=(1, MAX_SEQUENCE_LENGTH))

    x = LSTM(NUM_CELLS)(ip)
    x = Dropout(0.2)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)

    model.summary()

    # add load model code here to fine-tune

    return model

## Importing dataset

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_lofi_rowid_Nov13.csv')

We define a slicing windows and we are going to train the ltsm_fcn network on the sub sequences. 

In [ ]:
windows = 30

We define the subsquences, because the dataset is quite large we drop a row over two. 

In [ ]:
seq_list = [data.loc[i:(i+windows*2),'GR'].values for i in range(0,3500000,2)   ]

In [ ]:
y_list = [data.loc[i+windows,'label'] for i in range(0,3500000,2)   ]

The dataset is unbalanced so we use a class weight

In [ ]:
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_list)
                                               ,y_list)

In [ ]:
model = generate_lstmfcn(60, 5, NUM_CELLS=128)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
seq_len = 60
final_seq=sequence.pad_sequences(seq_list, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [ ]:
final_seq = np.array(final_seq)

### Splitting train and validation data set

In [ ]:
seq_train, seq_val, y_train, y_val = train_test_split(final_seq, pd.get_dummies(np.array(y_list)), test_size=0.2, random_state=42)


In [ ]:
seq_train =seq_train.reshape((seq_train.shape[0],1,seq_len))
seq_test =seq_test.reshape((seq_test.shape[0],1,seq_len))


An early stopping strategy is used in order to avoid overfitting.

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)

In [ ]:
model.fit(seq_train,y_train,epochs=20,validation_data=(seq_val,y_val),batch_size=126,callbacks=[es],class_weight =class_weight,verbose=2)

## Testing the model 

We test the model on the last pat of the dataset

In [ ]:
seq_list_test = [data.loc[i:(i+windows*2),'GR'].values for i in range(3600000,3800000,5)   ]

In [ ]:
y_list_test = [data.loc[i+windows,'label'] for i in range(3600000,3800000,5)   ]

In [ ]:
final_seq_test=sequence.pad_sequences(seq_list_test, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [ ]:
y_pred = model.predict(np.array([final_seq_test]).reshape((len(final_seq_test),1,60)))

In [ ]:
y_pred_label = np.argmax(y_pred,axis=1)

In [ ]:
confusion_matrix(y_list_test,y_pred_label)

In [ ]:
balanced_accuracy_score(y_list_test,y_pred_label)

In [ ]:
accuracy_score(y_list_test,y_pred_label)

## Predict on the real test set

In [ ]:
testDF = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_lofi_rowid_Nov13.csv')

In [ ]:
seq_list_test = [testDF.loc[i:(i+windows*2),'GR'].values for i in range(len(testDF)-windows*2)   ]

In [ ]:
final_seq_final =sequence.pad_sequences(seq_list_test, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [ ]:
y_pred = model.predict(np.array([final_seq_final]).reshape((len(final_seq_final),1,60)))

In [ ]:
y_pred_final = np.argmax(y_pred,axis=1)

In [ ]:
y_pred_final.shape

In [ ]:
testDF['label']=0

In [ ]:
testDF.loc[windows:len(testDF)-windows-1,'label'] = y_pred_final

In [ ]:
testDF[["row_id","well_id","label"]].to_csv("/content/drive/My Drive/Colab Notebooks/submission_lstm_fcn_5.csv",index =False)